## Sentiment change
##### Before runing this notebook, be sure to have the following two files before. Otherwise, these files can be created in the other notebooks
<ol>
  <li>json file containing both topics and sentiment</li>
  See: graph_from_sentiment_and_topic.ipynb

  <li>A csv-file containing information about the support for the parties</li>
  See: create_party_support_file.ipynb
</ol>


#### import graph file containing both sentiment and topic information

In [ ]:
# Written by Magnus Olander

from igraph import *
import json
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import matplotlib.patches as mpatches
import matplotlib.transforms as transforms
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from matplotlib.dates import date2num
from shapely.geometry import Point
import textwrap
import os

path_to_graph_file = ""

graph = Graph.Read_GraphML(path_to_graph_file + ".graphml")

nr_of_topics = len(set(v["topic_combined"] for v in graph.vs.select(type="question")))

print("The graph has",
      nr_of_topics,
      "topics that we will plot")

#### Fetching filtered party support data

###### Here you will need the file containing party support!

In [ ]:
path_to_polls_file = ""

polls = pd.read_csv(path_to_polls_file)

def support(year, month, party):
    
    if party == "Unfiliated":
        return 0.1
    
    result_df = polls[(polls['Year'] == year) & (polls['Month'] == month)]
    curr_support = result_df[party].values[0]
    
    return round(curr_support)

#### Help functions for creating sentiment-change plots

In [ ]:


def give_correct_party_name(party):
    
    if (party.lower() == "fp" or party.lower() == "l"):
        return "L"
    
    elif (party.lower() == "-" or party.lower() == "none" or party.lower() == None):
        return "Unfiliated"
    return party

def extract_data(graph, curr_category):
    test = []
    for vertex in graph.vs.select(type="question"):
        if vertex['topic_combined'] == curr_category:
            answerer = [give_correct_party_name(vertex["answer_by_party"]), vertex['answer_BERT_label'], vertex['question_date']]
            asker = [give_correct_party_name(vertex["question_by_party"]), vertex['question_BERT_label'], vertex['question_date']]
            test.append(asker)
            test.append(answerer)
    return test

def map_sentiments_to_colors():
    return {'POSITIVE': 'green', 'NEUTRAL': 'gray', 'NEGATIVE': 'red'}

def create_line_plot(ax, data, sentiment_colors, min_date, max_date):
    party_sentiments = defaultdict(list)

    for entry in data:
        party = entry[0]
        sentiment = entry[1]
        date_str = entry[2]
        date = datetime.strptime(date_str, '%Y-%m-%d')
        party_sentiments[party].append((date, sentiment))

    ax.margins(y=.1)

    # Plotting
    points = []
    for idx, (party, sentiments) in enumerate(party_sentiments.items()):
        sentiments.sort()
        current_color = None
        segment_start = sentiments[0][0]

        for date, sentiment in sentiments:
            if sentiment_colors[sentiment] != current_color:
                if current_color is not None:
                    ax.plot([segment_start, date], [party] * 2, color=current_color, markersize=5, linewidth=10)
                    add_support_text(ax, idx, segment_start, date, party, points)

                segment_start = date
                current_color = sentiment_colors[sentiment]

        if (segment_start + relativedelta(months=5) > max_date):
            end_date = max_date
        else:
            end_date = segment_start + relativedelta(months=5)
        ax.plot([segment_start, end_date], [party] * 2, color=current_color, markersize=5, linewidth=10)
        add_support_text(ax, idx, segment_start, end_date, party, points)

def add_support_text(ax, idx, start_date, end_date, party, points):
    support_x = start_date + (end_date - start_date) / 2
    y_position = idx
    support_y = y_position + 0.1
    party_support = support(end_date.year, end_date.month, party)
    point = Point(date2num(support_x), support_y)
    
    overlap = any(existing_point.y == point.y and existing_point.x - 250 < point.x < existing_point.x + 250 for existing_point in points)
    
    if not overlap:
        ax.annotate(f"{party_support}%", xy=(date2num(support_x), support_y),
                    xytext=(0, 3), textcoords='offset points',
                    va='bottom', ha='center')
        points.append(point)

def format_title(curr_category, min_date, max_date):
    formatted_title = re.sub(r'^\d+', '', curr_category.replace(',', ' ')).title()
    wrapped_title = textwrap.fill(formatted_title, width=2 * len(f"{min_date}-{max_date}"))
    return wrapped_title




#### Sentiment change function

In [ ]:

def plot_sentiment_graph(graph, highlight_periods, output_folder_name):
    unique_categories = set(v["topic_combined"] for v in graph.vs.select(type="question"))
    
    # Time period for the plot
    min_date = datetime(2006, 8, 1)
    max_date = datetime(2022, 9, 1)
    
    for curr_category in unique_categories:
        data = extract_data(graph, curr_category)
        sentiment_colors = map_sentiments_to_colors()

        fig, ax = plt.subplots(figsize=(15, 6))
        ax.xaxis_date()
        
        
        create_line_plot(ax, data, sentiment_colors, min_date, max_date)

        title = format_title(curr_category, min_date, max_date)
        plt.title(title, y=1.1, fontsize=18, fontweight='bold', fontstyle='italic', color='black', family='serif')

        plt.xlim(min_date, max_date)

        for start_date, end_date, color, label in highlight_periods:
            ax.axvspan(start_date, end_date, color=color, alpha=0.075)
            text_x = start_date + (end_date - start_date) / 2
            ax.text(text_x, ax.get_ylim()[1] + 0.3, label, ha='center', va='center', color='black', fontsize=8,
                    bbox=dict(facecolor='white', alpha=0.7))

        legend_labels = ['Positive', 'Neutral', 'Negative']
        legend_colors = ['green', 'grey', 'red', 'white']

        legend_patches = [mpatches.Patch(color=color, label=label) for color, label in zip(legend_colors, legend_labels)]

        legend = ax.legend(handles=legend_patches, title='Sentiment', loc='upper left', bbox_to_anchor=(1, 1),
                           facecolor='white', handlelength=7.5, handleheight=2, handletextpad=1)
        
        ### SAVE PLOTS
        filename = os.path.join(output_folder_name, f"plot_{curr_category[:12]}.png")
        plt.savefig(filename, bbox_inches='tight')
        plt.close()

#### Create the plots

In [ ]:
        

# Enter path to dicrionary to store the plots
output_folder_name = ""

# Period to highlight by the current ruling government. Red for left wing, blur for right wing
highlight_periods = [
    (datetime(2006, 10, 10), datetime(2014, 1, 2), "blue", "Reinfeldt"),
    (datetime(2014, 1, 3), datetime(2019, 1, 20), "red", "Löf. I"),
    (datetime(2019, 1, 21), datetime(2021, 7, 8), "red", "Löf. II"),
    (datetime(2021, 7, 9), datetime(2021, 11, 29), "red", "Löf. III"),
    (datetime(2021, 11, 30), datetime(2022, 9, 11), "red", "And.")
]

# Create Plots
plot_sentiment_graph(graph, highlight_periods, output_folder_name)